In [1]:
import spacy
from spacy import displacy
from spacy_lookup import Entity # use for dict NER
import pandas as pd
import glob
from importlib import reload

#### Ingredients

In [2]:
diary = ['butter','buttermilk','butterfat','casein','cheese','cream','creamer',
         'creme','curd','custard','dairy','dulce','egg','eggnog','gelato',
         'ghee','ice cream','kefir','milk','paneer','soft serve','sour cream',
         'whey','whey protein','whipped cream','whites','yogurt','yolks',
         'mayonnaise','mayo']

fruits = ['acai','apple','apples','apricot','apricots','banana','bananas',
          'berries','berry','blackberries','blackberry','blackcurrant',
          'blueberries','blueberry','cantaloupe','cherries','cherry',
          'clementine','clementines','coconut','coconuts','cranberries',
          'cranberry','currant','currants','date','dates','dragonfruit',
          'fig','figs','fruit','fruits','grape','grapefruit','grapes','guava',
          'honeydew','kiwi','kiwis','lemon','lemons','lime','limes','lychee',
          'mandarine','mandarines','mango','mangos','melon','melons',
          'nectarine','nectarines','olive','olives','orange','oranges',
          'papaya','papayas','passionfruit','peach','peaches','pear','pears',
          'pineapple','plum','plums','pomegranate','pomelo','pomelos','prune',
          'prunes','raisin','raisins','raspberries','raspberry','strawberries',
          'strawberry','tangerine','tangerines','watermelon']

grains = ['angel hair','baking powder','baking soda','barley','bread',
          'buckwheat','bulgur','bun','buns','campanelle','cappelletti',
          'cereal','cornmeal','couscous','crackers','dough','fettuccine',
          'flaxseed','flour','fusilli','grain','grano','grits','lasagna',
          'lasagne','linguine','loaf','macaroni','manicotti','millet','muesli',
          'muffins','muffins','noodle','noodles','oatmeal','oats','orzo',
          'pappardelle','pasta','penne','pita','popcorn','pretzel','pretzels',
          'quinoa','ravioli','rice','rigate','rigatoni','rocchetti','rolls',
          'rotini','rye','seitan','shells','spaghetti','spelt','tortellini',
          'tortilla','tortillas','vermicelli','yeast']

protein = ['abalone','anchovy','arctic char','bacon','barracuda','barramundi',
        'bass','bear','beef','beef heart','beef liver','beef tongue','bib',
        'bison','black drum','bluefish','bluenose','bone','breast','brisket',
        'buffalo','bullhead','butterfish','calf liver','calimari','capelin',
        'caribou','carp','catfish','caviar','char','chicken','chicken liver',
        'chilean seabass','chub','chuck','clam','cobia','cockle','cod','coho',
        'coley','conch','cornish game hen','corvina','crab','crab brown',
        'crawfish','crayfish','croaker','cusk','cuttle fish','dab','drum',
        'duck','duck liver','eel','emu','escargot','escolar','filet','fish',
        'flank','flounder','frog','gilthead','gizzards','goat','goose',
        'goose liver','greater amberjack','grenadier','grouper','grouse',
        'guinea hen','haddock','hake','halfmoon fish','halibut','ham',
        'harvest fish','herring','horse','imitation crab','jellyfish',
        'kangaroo','lamb','limpet slipper','lingcod','liver','lobster','loin',
        'mackerel','mahi-mahi','manila','marlin','marrow soup','meat',
        'monkfish','moose','mullet','muskellunge','mussel','mussels','mutton',
        'ocean pout','octopus','opah','opaleye','opossum','orange roughy',
        'organ meats','organs','ostrich','oyster','pangasius','parrotfish',
        'partridge','perch','pheasant','pickerel','pike','pilchards','plaice',
        'pollock','pollock, atlantic','pomfret','pompano','porgy','pork',
        'poultry','pouting','prawn','protein','pumpkinseed','quail','rabbit',
        'red snapper','rib','rockfish','rosefish','round','sablefish','salmon',
        'sanddabs','sardine','scad','scallops','scrod','scup','sea beam',
        'sea turtle','sea urchin','seabass','seafood','seatrout','shad',
        'shank','shark','sheepshead','shrimp','sirloin','skate','snail',
        'snake','sockeye','sole','spearfish','squab','squid','squirrel',
        'striped bass','sturgeon','sucker','sunfish','swai','sweetbreads',
        'swordfish','tatoaba','tenderloin','thigh','tilapia','tilefish',
        'tripe','trout','tuna','turbot','turkey','turtle','veal','venison',
        'wahoo','walleye','weakfish','whitefish','whiting','wing','wolfish',
        'wreckfish','yellowtail'] 

vegetables = ['artichoke','arugula','asparagus','avocado','bean','bean sprouts',
              'beans','beet','beetroot','beets','bell pepper','bell peppers',
              'bok choy','broccoli','brussels sprouts','butter beans',
              'butternut squash','cabbage','carrot','cauliflower','ceci beans',
              'celeriac','celery','chamomile','chard','chickpea','chickpeas',
              'chilis','collard greens','corn','cucumber','cucumbers','dill',
              'eggplant','fennel','garlic','greens','kale','leek','leeks',
              'legumes','lentils','lettuce','mushroom','mushrooms','onion',
              'peas','peppers','potato','potatoes','pumpkin','radicchio',
              'radish','radishes','scallion','scallions','shallot','shallots',
              'snap peas','spinach','sprouts','squash','tomato','tomatoes',
              'turnips','vegetable','yam','zucchini',] 

other = ['acorn','aioli','allspice','almond','almonds','anise','balm','basil',
         'bean','beans','bourbon','broth','capers','capsicum','cardamom',
         'cashew','cashews','chestnuts','chia','chicory','chimichurri','chives',
         'chocolate','chutney','cilantro','cinnamon','clove','cloves',
         'condiment','condiments','coriander','cumin','dijon','dill','dip',
         'dressing','drizzle','fats','flower','frosting','gin','ginger',
         'grease','guacamole','gum','habanero','hazelnut','herb','herbs',
         'hoisin','honey','horseradish','hummus','jalapeno','jam','jasmine',
         'jelly','ketchup','lard','lavender','leaf','lemongrass','licorice',
         'margarine','marigold','marmite','mayo','mint','miso',
         'mix','mole','mustard','myrtle','nut','nutella','nutmeg','nuts','oil',
         'oils','oregano','paprika','parsley','peanut','peanuts','pecan',
         'pecans','pepper','peppermint','pesto','pickle','pickles',
         'pico de gallo','pistachio','pistachios','poppy','powder','preserves',
         'reduction','relish','root','rosemary','rum','safflower','saffron',
         'sage','salsa','salt','sassafras','sauce','sauerkraut','seasoning',
         'seasonings','seed','seeds','sesame','shiso','shortening','spearmint',
         'spice','spices','spread','sriracha','sugar','sunflower','syrup',
         'tabasco','tahini','tarragon','tartar','thyme','tomato sauce',
         'turmeric','tzatziki','tzatzki','vanilla','vegemite','vinegar',
         'vodka','walnut','walnuts','wasabi','water','whisky','wine',
         'wintergreen','yeast','zest','ingredient','ingredients','lemon juice',
         'salad'] 

In [3]:
# print((diary))
# print((fruits))
# print((grains))
# print((protein))
# print((vegetables))
# print((other))

#### Tools

In [4]:
# Preparation Tools & Essentials
pte = ["Chef's Knife", 
'Chef Knife',       
'Cutting Board',
'Can Opener',
'Measuring Cups',
'Measuring Spoons',
'Mixing Bowls',
'Colander',
'Vegetable Peeler',
'Potato Masher',
'Whisk',
'Salad Spinner',
'Grater',
'Shears',
'Citrus Juicer',
'Garlic Press',
'Paring Knife',
'Bread Knife',
'Honing Ceramic Rod',
'Sharpening Ceramic Rod',
'Knife Sharpener']

# Cookware & Bakeware
cbware = ['Stainless Steel Skillet',
'Sauté Pan',
'Small Saucepan',
'Medium Saucepan',
'Large Pot',
'Cast Iron Skillet',
'Grill Pan',
'Baking Sheet Pan',
'Casserole Dish',
'Broiler Pan',
'Stockpot']

# Cooking Tools & Kitchen Utensils
ctku = ['Spatula',
'Stirring Spoon',
'Tongs',
'Ladle',
'Oven Mitts',
'Trivet',
'Splatter Guard',
'Thermometer']

# Appliances
appliances = ['Immersion Blender',
             'Kitchen Scale',
             'Blender']

# Kitchen essentials
ke = ['Food Storage Containers',
'Aluminum Foil',
'Parchment Paper',
'Towels',
'Sponges',
'Heavy Sponges',
'Dish Rack',
'Ice Cube Tray',
'Trash Bags',
'Trash Bin']

In [5]:
# print(pte)
# print(ctku)
# print(appliances)
# print(ke)

#### Actions

In [6]:
actions = ['Arrange',
'Baste',
'Beat',
'Blend',
'Brown',
'Build',
'Bury',
'Carve',
'Check',
'Chop',
'Close',
'Cool',
'Correct',
'Cover',
'Crumple',
'Cut',
'Decorate',
'Discard',
'Divide',
'Drape',
'Drop',
'Dry',
'Film',
'Fold',
'Follow',
'Form',
'Force',
'Glaze',
'Insert',
'Lay',
'Leave',
'Lift',
'Make',
'Melt',
'Mince',
'Mix',
'Moisten',
'Mound',
'Open',
'Pack',
'Paint',
'Pierce',
'Pour',
'Prepare',
'Press',
'Prick',
'Pull',
'Puree',
'Push',
'Quarter',
'Raise',
'Reduce',
'Refresh',
'Reheat',
'Replace',
'Return',
'Ring',
'Roast',
'Roll',
'Salt',
'Saute',
'Scatter',
'Scoop',
'Scrape',
'Scrub',
'Season',
'Separate',
'Set',
'Settle',
'Shave',
'Simmer',
'Skim',
'Slice',
'Slide',
'Slip',
'Slit',
'Smear',
'Soak',
'Spoon',
'Spread',
'Sprinkle',
'Stir',
'Strain',
'Strew',
'Stuff',
'Surround',
'Taste',
'Thin',
'Tie',
'Tilt',
'Tip',
'Top',
'Toss',
'Trim',
'Turn',
'Twist',
'Warm',
'Wilt',
'Wind',
'Wrap']

In [7]:
#print(actions)

In [8]:
ingredients = diary + fruits + grains + vegetables + other
tools = pte + ctku + appliances + ke

In [9]:
nlp = spacy.load('en')

# add ingredients tags
ingredients_e = Entity(keywords_list=ingredients, label='INGRD')

# add tools tags
tools_e = Entity(keywords_list=tools, label='TOOL')

# add actions tags
actions_e = Entity(keywords_list=actions, label='ACTION')

# add to pipeline
nlp.add_pipe(actions_e, last=True, name="Action")
nlp.add_pipe(ingredients_e, last=True, name="Ingredient")
nlp.add_pipe(tools_e, last=True, name="Tool")

for name in nlp.pipeline:
    print(name)

('tagger', <spacy.pipeline.Tagger object at 0x7f4a83f98320>)
('parser', <spacy.pipeline.DependencyParser object at 0x7f4a82e9c8e0>)
('ner', <spacy.pipeline.EntityRecognizer object at 0x7f4a82e9c938>)
('Action', <spacy_lookup.Entity object at 0x7f4a84ce6668>)
('Ingredient', <spacy_lookup.Entity object at 0x7f4a84ce6208>)
('Tool', <spacy_lookup.Entity object at 0x7f4a84ce6240>)


In [10]:
# bring 1 json into a dataframe for analysis
files = sorted(glob.glob('../layer1.json.[5].valid.gz'))
df = pd.concat([pd.read_pickle(fp) for fp in files], ignore_index=True)
df.shape

(100000, 9)

In [11]:
# for simplicity, let's filter out epicurious recipes with less than 5 steps
keep_columns = ['id', 'partition', 
                'collection', 'title', 
                'ingr_count', 'instr_count', 
                'ingredients', 'instructions']

epicurious = df[(df.collection == 'www.epicurious.com') 
   & (df.instr_count<=5)][keep_columns].sort_values(by=['instr_count'], ascending=False)

# train, val, test counts
epicurious.groupby('partition').size()

partition
test     111
train    500
val      112
dtype: int64

In [12]:
recipes = epicurious[(epicurious.partition=='train')].index
len(recipes)

500

#### uncomment to create json line file

In [13]:
# import json
# exclude_these = [
# 'PERSON',
# 'NORP',
# 'FAC',
# 'ORG',
# 'GPE',
# 'LOC',
# 'PRODUCT',
# 'EVENT',
# 'WORK_OF_ART',
# 'LAW',
# 'LANGUAGE',
# 'MONEY',
# 'ORDINAL', 
# 'DATE',
# 'PERCENT',    
# 'CARDINAL'    
# ]

# nlp = spacy.load('en')

# ingredients_e = Entity(keywords_list=ingredients, label='INGRD')
# tools_e = Entity(keywords_list=tools, label='TOOL')
# actions_e = Entity(keywords_list=actions, label='ACTION')

# nlp.add_pipe(actions_e, last=True, name="Action")
# nlp.add_pipe(ingredients_e, last=True, name="Ingredient")
# nlp.add_pipe(tools_e, last=True, name="Tool")

# jsonl = open("../KH_Data/test_2_epicurious_KH_original.jsonl", "w")
# line = {}

# for recipe in recipes: 
#     for each in epicurious['instructions'][recipe]:
        
#         try:
#             # block raising an exception
#             for instruction in list(each.values()):
#                 doc = nlp(instruction)
#                 line['text'] = doc.text
#                 entities = []

#             if (len(doc.ents) > 0):
#                 for i in range(len(doc.ents)):
#                     if doc.ents[i].label_ not in exclude_these:
#                         entities.append({"start" : doc.ents[i].start_char,
#                                          "end" : doc.ents[i].end_char,
#                                          'label': doc.ents[i].label_ 
#                                         })
#             line['spans'] = entities
#             line['answer'] = 'reject'
#             jsonl.writelines(json.dumps(line) + "\n")

#         except:
#             pass # doing nothing on exception
        
# jsonl.close()

#### iteration 2, boosted seeding 

In [14]:
more_tools = [
    'aluminum foil','amphora','applicator','aspirator','auger','balisong','bandsaw',
    'beater','blender','blenders','botle','bowl','bowls','bread knife','breadbox','breadmaker',
    'breville','brush','caddy','cafetiere','caldron','can opener','canners','caserole','cellophane',
    'cellotape','chef knife',"chef's knife",'chisels','chopstick','chopsticks','citrus juicer','cleavers','coffeemaker','coffeepot','colander','container','cosies','crimper','crimpers','crispers','crock','crockery','crockpot','cups','cutco','cutter','cutters','cutting board','demitasse','depressors','detergent','dipper','dish rack','dishcloths','dishes','dishware','dishwasher','dishwashers','disperser','disposer','dremel','drinkware','drip','drippers','dropper','electrolux','enamelware','ewers','exacto','fireplace','fireplaces','flagons','flosser','food storage containers','fork','forschner','freezer','frier','frothers','garlic press','glas','glassware','glue','goblets','grater','griddle','griddler','hand','handkerchief','hankies','heavy sponges','hemostat','hemostats','honing ceramic rod','hopper','hotpoint','ice cube tray','immersion blender','jar','joiner','jointer','juicer','kcup','kilner','kitchen scale','knife','knife sharpener','knive','ladel','ladle','ladles','lid','linens','makita','mashers','matchbooks','mazzer','measuring cups','measuring spoons','microwave','miele','mincer','minibar','mircowave','miter','mixer','mixers','mixing bowls','moulder','muddler','mugs','mylar','napkin','nesco','oven mitts','overcap','paintbrush','papertowel','parasols','parchment paper','paring knife','percolator','percolators','phials','pipecleaners','planer','plasic','plastic','plastics','polisher','portafilter','pot','potato masher','potholders','pruners','pyrex','refigerator','refridgerators','refrig','refrigerator','salad spinner','sandblaster','santoku','saucepan','saucer','sawzall','scalpels','scissors','scoville','shaker','sharpener','sharpening ceramic rod','shears','sheeter','sieve','sifter','sippies','skillets','slicer','slicers','slitter','slotter','snips','spacemaker','spatula','splatter guard','sponge','sponges','spoon','spoons','sprayer','squeezer','staplers','stein','steins','stemware','stick','stirrers','stirring spoon','stix','stopper','stove','stoves','stovetops','straws','sunbeam','switchblade','switchblades','teaware','thermometer','tinplate','toiletries','tongs','toothbrush','toothbrushes','toothpick','toweling','towels','towles','trash bags','trash bin','trays','trivet','trivets','tweezers','utensil','utensils','vase','vases','vegetable peeler','waffler','washcloth','washclothes','washcloths','washrag','waterbath','wax','whipper','whisk','whisks','wisk','wok','xacto','ziploc','ziplocs']

In [15]:
more_actions = ['build','roll','press','surround','toss','correct','mix','discard','thin','melt','warm','fold','return','shave','spoon','divide','carve','stuff','insert','rrim','taste','drop','prepare','cool','moisten','sprinkle','skim','lift','refresh','scoop','film','decorate','pour','reduce','wilt','glaze','check','follow','pierce','set','lay','prick','strain','scatter','push','slide','beat','soak','simmer','scrape','season','blend','paint','tilt','crumple','cover','turn','close','mince','force','strew','bury','form','tip','raise','stir','pack','settle','drape','dry','saute','slit','quarter','brown','slice','ring','smear','scrub','puree','wrap','arrange','top','wind','pull','separate','twist','chop','mound','salt','reheat','roast','make','leave','slip','tie','baste','replace','spread','cut','open','pull','slice','wilt','shave','cool','tie','scrape','sprinkle','slit','roast','surround','replace','tilt','lay','film','roll','wrap','bury','saute','carve','toss','separate','crumple','spread','strain','mince','check','salt','cover','scoop','twist','prepare','refresh','smear','insert','settle','trim','stuff','brown','stir','pack','glaze','raise','slip','fold','discard','thin','wind','tip','correct','beat','mix','soak','turn','mound','simmer','blend','make','drape','quarter','open','reheat','return','strew','paint','set','prick','slide','reduce','moisten','skim','season','decorate','force','lift','divide','leave','close','ring','follow','dry','scrub','pierce','cut','scatter','press','puree','melt','form','spoon','chop','baste','arrange','drop','pour','taste','top','push','warm','build','thin','strain','drape','scrape','dry','soak','divide','prick','replace','tip','warm','roast','twist','discard','form','cut','crumple','spread','surround','scrub','skim','scoop','mound','follow','press','blend','build','strew','shave','refresh','push','decorate','slip','taste','bury','cool','fold','arrange','toss','pull','trim','lay','paint','prepare','slice','simmer','ring','film','smear','pack','tie','leave','slit','melt','close','reduce','stuff','glaze','puree','pour','roll','separate','saute','turn','mince','moisten','return','scatter','season','chop','set','mix','quarter','raise','brown','top','check','insert','carve','drop','make','beat','force','wrap','cover','tilt','pierce','correct','lift','wind','salt','slide','spoon','baste','settle','sprinkle','open','wilt','stir','reheat','scoop','baste','lay','scatter','push','wrap','surround','mound','prepare','make','set', 'peel']

In [16]:
nlp = spacy.load('en')

# add ingredients tags
ingredients_e = Entity(keywords_list=ingredients, label='INGRD')

# add tools tags
tools_e = Entity(keywords_list=more_tools, label='TOOL')

# add actions tags
actions_e = Entity(keywords_list=more_actions, label='ACTION')

# add to pipeline
nlp.add_pipe(actions_e, last=True, name="Action")
nlp.add_pipe(ingredients_e, last=True, name="Ingredient")
nlp.add_pipe(tools_e, last=True, name="Tool")

In [17]:
displacy.render(nlp('peel an apple and put it into the bowl'), style='ent', jupyter=True)

In [18]:
import pandas as pd

In [19]:
df = pd.read_json('../KH_Data/test_2_epicurious_KH_original.jsonl', lines=True)
df.columns

Index(['answer', 'spans', 'text'], dtype='object')

#### uncomment to create json line file

In [20]:
# import json 

# exclude_these = [
# 'PERSON',
# 'NORP',
# 'FAC',
# 'ORG',
# 'GPE',
# 'LOC',
# 'PRODUCT',
# 'EVENT',
# 'WORK_OF_ART',
# 'LAW',
# 'LANGUAGE',
# 'MONEY',
# 'ORDINAL', 
# 'DATE',
# 'PERCENT',    
# 'CARDINAL'    
# ]

# jsonl = open("../KH_Data/test_2_epicurious_KP_original_annot2.jsonl", "w")
# line = {}

# for each in df.text:
#     try:
#         # block raising an exception
#         doc = nlp(each)
#         line['text'] = doc.text
#         entities = []

#         if (len(doc.ents) > 0):
#             for i in range(len(doc.ents)):
#                 if doc.ents[i].label_ not in exclude_these:
#                     entities.append({"start" : doc.ents[i].start_char,
#                                      "end" : doc.ents[i].end_char,
#                                      'label': doc.ents[i].label_ 
#                                     })
#         line['spans'] = entities
#         line['answer'] = 'reject'
#         jsonl.writelines(json.dumps(line) + "\n")

#     except:
#         pass # doing nothing on exception

# jsonl.close()